In [ ]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
import shutil
import datetime
from datetime import datetime,date
import re
import json
from pandas.api.types import is_numeric_dtype

# from simple_salesforce import Salesforce, format_soql, format_external_id
# from simple_salesforce.exceptions import SalesforceMalformedRequest
# import simpler_sf
from sqlalchemy import create_engine
#import pymysql

# from mysql import connector


# paramiko.util.log_to_file("paramiko.log")
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")

In [ ]:
# usuario = 'siempreread'
# contrasena = 'S13mpr3_Ch1l3'
# url_servidor = '209.151.148.250'
puerto = "3306"
esquema = "MetLife_A"
plugin_autenticacion = "mysql_native_password"

usuario = "care_consulta"
contrasena = "CareAssistance!10"
url_servidor = "10.0.10.22"

# Conexión a la base de datos

c_conexion = "mysql+mysqlconnector://{0}:{1}@{2}:{3}/{4}?auth_plugin={5}"
c_conexion = c_conexion.format(
    usuario, contrasena, url_servidor, puerto, esquema, plugin_autenticacion
)
motor_mysql_mariadb = create_engine(c_conexion)

#table.to_sql(name='rut_unico_tiempo',con=motor_mysql_mariadb,if_exists = 'append')


# archivo de salida modificar segun archivo
# archivo = 'C:/Users/jmartinez/Desktop/casos nuevos/clientes/1/casos_faltantes_segun_archivo.xlsx'

In [ ]:
# table.to_sql(name='prueba',con=motor_mysql_mariadb,if_exists = 'append')
rut_unicos = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unicos WHERE LEFT(Rut_T, 1) <> '0' "
rut_unicos = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("-", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace("–", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].astype(str)
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(".", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.replace(" ", "")
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.upper()
rut_unicos["Rut_C"] = rut_unicos["Rut_C"].str.strip()

rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("-", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace("–", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].astype(str)
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(".", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.replace(" ", "")
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.upper()
rut_unicos["Rut_T"] = rut_unicos["Rut_T"].str.strip()

rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(",", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace(".", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("-", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┐", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("═", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("╔", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.replace("┌", "")
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.title()
rut_unicos["Nombre"] = rut_unicos["Nombre"].str.strip()

rut_unicos.shape



""" rut_unicos.to_excel(
    "F:/Empresas SF/reportes/rut_unicos_hasta_Marzo_2024.xlsx", index=False
) """

In [12]:
rut_unico_tiempo_ = pd.DataFrame()
sql = "SELECT * FROM MetLife_A.rut_unico_tiempo_2 "
rut_unico_tiempo_ = pd.read_sql_query(sql, con=motor_mysql_mariadb)

rut_unico_tiempo_.shape

(5892938, 11)

In [ ]:
def sacarultimo(y):
    y = y[:-1]
    return str(y)
    

In [ ]:
sftp = "F:/2024/MetLife/metlife_analisis_todo.xlsx"

derty = pd.read_excel(sftp)

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut"].apply(lambda y: sacarultimo(y))

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].astype(str)
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.upper()
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace("-", "")
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace("–", "")
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].astype(str)
rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.replace(".", "")

rut_unico_tiempo_["rut_rut2"] = rut_unico_tiempo_["rut_rut2"].str.strip()

derty["rut1"] = derty["rut1"].astype(str)
derty["rut1"] = derty["rut1"].str.upper()
derty["rut1"] = derty["rut1"].str.replace("-", "")
derty["rut1"] = derty["rut1"].str.replace("–", "")
derty["rut1"] = derty["rut1"].astype(str)
derty["rut1"] = derty["rut1"].str.replace(".", "")

derty["rut1"] = derty["rut1"].str.strip()


In [ ]:
rut_unicos2 = pd.merge(
    left=derty,
    right=rut_unico_tiempo_,
    how="left",
    left_on="rut1",
    right_on="rut_rut2",
)


In [ ]:
rut_unicos2.shape

In [ ]:
rut_unicos2.head()

In [ ]:
rut_unicos2 = rut_unicos2.drop_duplicates(
    [
        "rut_rut2","canal_rut","fecha_rut","poliza_rut",
    ],
    keep="last",
)
rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].astype(str)

rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].str.replace("Q", "sftp")
rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].str.replace("correo2", "correo")
rut_unicos2["canal_rut"] = rut_unicos2["canal_rut"].str.replace("email", "correo")


In [ ]:
enero2023 = pd.DataFrame()
enero2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023")]
enero2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
enero2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
enero2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

febrero2023 = pd.DataFrame()
febrero2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023")]
febrero2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
febrero2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
febrero2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

marzo2023 = pd.DataFrame()
marzo2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023")]
marzo2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
marzo2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
marzo2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]


abril2023 = pd.DataFrame()
abril2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023")]
abril2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
abril2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
abril2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "4") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

mayo2023 = pd.DataFrame()
mayo2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023")]
mayo2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
mayo2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
mayo2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "5") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

junio2023 = pd.DataFrame()
junio2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023")]
junio2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
junio2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
junio2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "6") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

julio2023 = pd.DataFrame()
julio2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023")]
julio2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
julio2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
julio2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "7") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

agosto2023 = pd.DataFrame()
agosto2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023")]
agosto2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
agosto2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
agosto2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "8") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

septiembre2023 = pd.DataFrame()
septiembre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023")]
septiembre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
septiembre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
septiembre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "9") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

octubre2023 = pd.DataFrame()
octubre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023")]
octubre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
octubre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
octubre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "10") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

noviembre2023 = pd.DataFrame()
noviembre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023")]
noviembre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
noviembre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
noviembre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "11") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]

diciembre2023 = pd.DataFrame()
diciembre2023 = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023")]
diciembre2023W = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "web") ]
diciembre2023S = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "sftp") ]
diciembre2023C = rut_unicos2[(rut_unicos2["mes_rut"] == "12") & (rut_unicos2["año_rut"] == "2023") & (rut_unicos2["canal_rut"] == "correo") ]


enero2024 = pd.DataFrame()
enero2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024")]
enero2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
enero2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
enero2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "1") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

febrero2024 = pd.DataFrame()
febrero2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024")]
febrero2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
febrero2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
febrero2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "2") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

marzo2024 = pd.DataFrame()
marzo2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024")]
marzo2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
marzo2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
marzo2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

abril2024 = pd.DataFrame()
abril2024 = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024")]
abril2024W = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "web") ]
abril2024S = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "sftp") ]
abril2024C = rut_unicos2[(rut_unicos2["mes_rut"] == "3") & (rut_unicos2["año_rut"] == "2024") & (rut_unicos2["canal_rut"] == "correo") ]

#Minera_Abra = Minera_Abra[(Minera_Abra["Empleador"] == Minera_Abra["Nombre Empresa"])]

In [ ]:
enero = pd.DataFrame()
enero = derty.copy()
enero["Enero"]  = enero["rut1"].isin(enero2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero web"]  = enero["rut1"].isin(enero2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero sftp"]  = enero["rut1"].isin(enero2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero correo"]  = enero["rut1"].isin(enero2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Febrero"]  = enero["rut1"].isin(febrero2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero web"]  = enero["rut1"].isin(febrero2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero sftp"]  = enero["rut1"].isin(febrero2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero correo"]  = enero["rut1"].isin(febrero2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Marzo"]  = enero["rut1"].isin(marzo2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo web"]  = enero["rut1"].isin(marzo2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo sftp"]  = enero["rut1"].isin(marzo2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo correo"]  = enero["rut1"].isin(marzo2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Abril"]  = enero["rut1"].isin(abril2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril web"]  = enero["rut1"].isin(abril2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril sftp"]  = enero["rut1"].isin(abril2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril correo"]  = enero["rut1"].isin(abril2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Mayo"]  = enero["rut1"].isin(mayo2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo web"]  = enero["rut1"].isin(mayo2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo sftp"]  = enero["rut1"].isin(mayo2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Mayo correo"]  = enero["rut1"].isin(mayo2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Junio"]  = enero["rut1"].isin(junio2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Junio web"]  = enero["rut1"].isin(junio2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Junio sftp"]  = enero["rut1"].isin(junio2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Junio correo"]  = enero["rut1"].isin(junio2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Julio"]  = enero["rut1"].isin(julio2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Julio web"]  = enero["rut1"].isin(julio2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Julio sftp"]  = enero["rut1"].isin(julio2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Julio correo"]  = enero["rut1"].isin(julio2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Agosto"]  = enero["rut1"].isin(agosto2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Agosto web"]  = enero["rut1"].isin(agosto2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Agosto sftp"]  = enero["rut1"].isin(agosto2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Agosto correo"]  = enero["rut1"].isin(agosto2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Septiembre"]  = enero["rut1"].isin(septiembre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Septiembre web"]  = enero["rut1"].isin(septiembre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Septiembre sftp"]  = enero["rut1"].isin(septiembre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Septiembre correo"]  = enero["rut1"].isin(septiembre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Octubre"]  = enero["rut1"].isin(octubre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Octubre web"]  = enero["rut1"].isin(octubre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Octubre sftp"]  = enero["rut1"].isin(octubre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Octubre correo"]  = enero["rut1"].isin(octubre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Noviembre"]  = enero["rut1"].isin(noviembre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Noviembre web"]  = enero["rut1"].isin(noviembre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Noviembre sftp"]  = enero["rut1"].isin(noviembre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Noviembre correo"]  = enero["rut1"].isin(noviembre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Diciembre"]  = enero["rut1"].isin(diciembre2023["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Diciembre web"]  = enero["rut1"].isin(diciembre2023W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Diciembre sftp"]  = enero["rut1"].isin(diciembre2023S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Diciembre correo"]  = enero["rut1"].isin(diciembre2023C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Enero 2024"]  = enero["rut1"].isin(enero2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero 2024 web"]  = enero["rut1"].isin(enero2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero 2024 sftp"]  = enero["rut1"].isin(enero2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Enero 2024 correo"]  = enero["rut1"].isin(enero2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Febrero 2024"]  = enero["rut1"].isin(febrero2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero 2024 web"]  = enero["rut1"].isin(febrero2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero 2024 sftp"]  = enero["rut1"].isin(febrero2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Febrero 2024 correo"]  = enero["rut1"].isin(febrero2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Marzo 2024"]  = enero["rut1"].isin(marzo2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo 2024 web"]  = enero["rut1"].isin(marzo2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo 2024 sftp"]  = enero["rut1"].isin(marzo2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Marzo 2024 correo"]  = enero["rut1"].isin(marzo2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

enero["Abril 2024"]  = enero["rut1"].isin(marzo2024["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril 2024 web"]  = enero["rut1"].isin(marzo2024W["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril 2024 sftp"]  = enero["rut1"].isin(marzo2024S["rut_rut2"]).astype(str).replace("False","").replace("True","X")
enero["Abril 2024 correo"]  = enero["rut1"].isin(marzo2024C["rut_rut2"]).astype(str).replace("False","").replace("True","X")

In [ ]:
enero.head()

In [ ]:
enero.shape

In [ ]:
enero.to_excel("F:/2024/MetLife/datos_metlife_fecha_canal.xlsx", index=False)

In [ ]:
enerob = enero[[
    "rut1","VIGENTE 19.03.2023","EN WEBSERVICE","OBSERVACION","Enero","Febrero","Marzo","Abril","Mayo","Junio","Julio","Agosto","Septiembre","Octubre","Noviembre","Diciembre","Enero 2024","Febrero 2024","Marzo 2024"
]]

In [ ]:
enerob.head()

In [ ]:
enerob.to_excel("F:/2024/MetLife/datos_metlife_fecha_mes.xlsx", index=False)

In [ ]:
derty["esta"] = derty["rut1"].isin(rut_unicos2["rut_rut2"])
rut_unicos["rut_sin"] = rut_unicos["Rut_T"].apply(lambda y: sacarultimo(y))
derty["unicos"] = derty["rut1"].isin(rut_unicos["rut_sin"])
derty.to_excel("F:/2024/MetLife/datos_metlife_esta.xlsx", index=False)


In [13]:
def fechaValida(fecha):
    if (len(str(fecha)) <= 9 ):
        fecha = ""
        
    fecha = str(fecha)
    fecha1 = str(fecha)
    
    #datetime.strptime(fecha, '%Y-%m-%d')
    
    try:
        fecha = datetime.strptime(fecha, '%Y-%m-%d %H:%M:%S')
        date_string = fecha1
        #print("date_string =", date_string)
        #print("type of date_string =", type(date_string))

        date_object = datetime.strptime(date_string, "%Y-%m-%d %H:%M:%S")

        #print("date_object =", date_object)
        #print("type of date_object =", type(date_object))
        return date_object
    except ValueError:
        return datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

In [14]:
mes = 4
año = "2024"
fecha = año + "-" + str(mes) + "-" + "01"  
mesesDic = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}

str(mesesDic[mes])



sftp = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/sftp.xlsx"


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"


derty = pd.read_excel(sftp)

In [15]:
derty.shape

(326270, 24)

In [16]:
metlife_sftp = pd.DataFrame()
metlife_sftp = derty.copy()

In [17]:


metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("-", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace("–", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].astype(str)
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(" ", "")
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.upper()
metlife_sftp["rut1"] = metlife_sftp["rut1"].str.strip()

metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("-", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace("–", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].astype(str)
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(" ", "")
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.upper()
metlife_sftp["rut2"] = metlife_sftp["rut2"].str.strip()

metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(",", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace(".", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("-", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┐", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("═", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("╔", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.replace("┌", "")
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.title()
metlife_sftp["Nombre"] = metlife_sftp["Nombre"].str.strip()

metlife_sftp["poliza"] = metlife_sftp["poliza"].astype(str)
metlife_sftp["poliza"] = metlife_sftp["poliza"].str.strip()



#*******************sacar primer cero
metlife_sftp.shape

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\3662042864.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut1"] = metlife_sftp["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\3662042864.py:16: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["rut2"] = metlife_sftp["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\3662042864.py:22: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_sftp["Nombre"] = metlife_sftp["Nomb

(326270, 24)

In [18]:
metlife_sftp.head()

,poliza,RUTASEGURADO,DVASEGURADO,rut2,RUTBENEFICIARIO,DVBENEFICIARIO,rut1,sdfsdfdsf,Unnamed: 8,RUTBENEFIARIO_DV,...,CIUDAD,TELEFONO,RELACION,desde,hasta,Nombre,TITULAR,EMAIL,CELULAR,VIGENCIA_HASTA
0,340023670,18434052,6,184340526,18434052,6,184340526,1,184340526,6,...,NaN,NaN,AS,2023-01-01,NaT,Riquelme Arias Juan Alejandro,Riquelme Arias Juan Alejandro,NaN,NaN,45657.0
1,340022020,18066094,1,180660941,18066094,1,180660941,1,180660941,1,...,SANTIAGO,NaN,AS,2022-08-01,NaT,Zepeda Hernandez Constanza Andrea,Zepeda Hernandez Constanza Andrea,CONSTANZA_ZEPEDA@LIVE.CL,979949391,45596.0
2,340002458,17085671,6,170856716,17085671,6,170856716,1,170856716,6,...,SANTIAGO,NaN,AS,2023-09-01,NaT,Sandoval Salinas Jose Pablo,Sandoval Salinas Jose Pablo,jose.sandoval@latam.com,976093700,45504.0
3,340022865,16097460,5,160974605,16097460,5,160974605,1,160974605,5,...,NaN,NaN,AS,2022-12-01,NaT,Espinoza Orostica Maximiliano Ricardo,Espinoza Orostica Maximiliano Ricardo,THE.OROSTICA@GMAIL.COM,965970153,45626.0
4,340023194,8563527,1,85635271,8563527,1,85635271,1,85635271,1,...,NaN,56940214308,AS,2023-01-01,NaT,Perez Cifuentes Domingo Luis,Perez Cifuentes Domingo Luis,dperezxtreme@gmail.com,40214308,45657.0


In [19]:
def siPrimerCero(y):    
    if len(y) <= 4:
        y =  "NONE"
    if y[0] == "0":
        y = y[1:]
    return str(y)
    

In [20]:
metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_sftp = metlife_sftp[(metlife_sftp["rut1"] != "NONE")]
metlife_sftp = metlife_sftp[(metlife_sftp["rut2"] != "NONE")]


metlife_sftp["Titular"] = np.where(metlife_sftp["rut1"] == metlife_sftp["rut2"],1,2 )
metlife_sftp = metlife_sftp[(metlife_sftp["Titular"] == 1)]
metlife_sftp = metlife_sftp.drop_duplicates(
    [
        "rut1","poliza",
    ],
    keep="last",
)

metlife_sftp.shape

(326268, 25)

In [21]:
metlife_sftp.shape

(326268, 25)

In [22]:
correo = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/correo.xlsx"
dertyc = pd.read_excel(correo)
metlife_correo = dertyc.copy()

# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("-", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace("–", "")
metlife_correo["rut1"] = metlife_correo["rut1"].astype(str)
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(" ", "")
metlife_correo["rut1"] = metlife_correo["rut1"].str.upper()
metlife_correo["rut1"] = metlife_correo["rut1"].str.strip()

metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("-", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace("–", "")
metlife_correo["rut2"] = metlife_correo["rut2"].astype(str)
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(" ", "")
metlife_correo["rut2"] = metlife_correo["rut2"].str.upper()
metlife_correo["rut2"] = metlife_correo["rut2"].str.strip()

metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(",", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace(".", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("-", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┐", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("═", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("╔", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.replace("┌", "")
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.title()
metlife_correo["Nombre"] = metlife_correo["Nombre"].str.strip()

metlife_correo["poliza"] = metlife_correo["poliza"].astype(str)
metlife_correo["poliza"] = metlife_correo["poliza"].str.strip()




#*******************sacar primer cero
metlife_correo.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\2005110109.py:14: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut1"] = metlife_correo["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\2005110109.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["rut2"] = metlife_correo["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\2005110109.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_correo["Nombre"] = metlife

(40, 15)

In [23]:
metlife_correo["rut1"] = metlife_correo["rut1"].apply(lambda y: siPrimerCero(y))
metlife_correo["rut2"] = metlife_correo["rut2"].apply(lambda y: siPrimerCero(y))
metlife_correo = metlife_correo[(metlife_correo["rut1"] != "NONE")]
metlife_correo = metlife_correo[(metlife_correo["rut2"] != "NONE")]

metlife_correo["Titular"] = np.where(metlife_correo["rut1"] == metlife_correo["rut2"],1,2 )
metlife_correo = metlife_correo[(metlife_correo["Titular"] == 1)]
metlife_correo = metlife_correo.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)

metlife_correo.shape

(17, 16)

In [ ]:
metlife_correo.shape

In [24]:
web = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/web.xlsx"
dertyd = pd.read_excel(web)


In [25]:
dertyd.head(9)

,id,ciudad,codigo_carga,codigo_postal,comuna,convenio,convenio_plan,direccion,hasta,fecha_modificacion,...,p_paterno,Nombre,poliza,p_relacion,rut1,rut2,p_sexo,procesado,rut_empresa,version
0,515658,ANTOFAGASTA,0,NaN,VILLA SAN LORENSO,77,SIN PLAN,DON MILENKO 305-A,0,2024-03-01,...,OLIVARES,IVAN OLIVARES ASTORGA,340017678,1,80219598,80219598,1,0,774185801,1
1,515659,NaN,0,NaN,CALAMA,77,SIN PLAN,SAN FERNANDO 1252,0,2024-03-01,...,MUÑOZ,RODRIGO ALEXIS MUÑOZ SOTO,340023043,1,151477666,151477666,1,0,785903609,1
2,515661,SANTIAGO,0,NaN,EL BOSQUE,77,SIN PLAN,LAS PARCELAS 1927,0,2024-03-01,...,CABEZAS,JAZMIN ALEJANDR CABEZAS ABARZUA,340024817,1,19830249K,19830249K,2,0,760846821,1
3,515665,NaN,0,NaN,MACHALÍ,77,SIN PLAN,RECREO N 100,0,2024-03-01,...,GUTIERREZ,JUAN ANTONIO GUTIERREZ CERON,340023194,1,142043572,142043572,1,0,969537001,1
4,515667,NaN,0,NaN,LOS ANDES,77,SIN PLAN,PB PASO EL BERMEJO ALGARROBO 1,0,2024-03-01,...,QUINAN,CLAUDIO MARCELO QUINAN QUINAN,340023194,1,113104635,113104635,1,0,969537001,1
5,515670,NaN,0,NaN,NaN,77,SIN PLAN,VILLA SAN LUIS PASAJE 1 CASA 290,0,2024-03-01,...,PEREIRA,GUILLERMO PEREIRA GAVILAN,340000644,1,117662500,117662500,1,0,93546000K,1
6,515672,NaN,0,NaN,VALPARAÍSO,80,SIN PLAN,IVAN LU CSIC CALLE 6 CASA 1 C PLACERES,0,2024-03-01,...,QUINONES,IGNACIO QUINONES ZUBELDIA,340010579,1,197738766,197738766,1,0,966026405,1
7,515674,NaN,0,NaN,LA SERENA,80,SIN PLAN,ALEJANDRO FLORES 1714 LA SERENA,0,2024-03-01,...,ROJAS,NICOLAS FRANCIS ROJAS ROJAS,340019003,1,179799391,179799391,1,0,789216908,1
8,515676,NaN,0,NaN,PADRE HURTADO,80,SIN PLAN,SEVILLA 2075 INDEPENDIENTE,0,2024-03-01,...,BRIONES,CAMILA BRIONES DÍAZ,340019003,1,18833170K,18833170K,2,0,789216908,1


In [26]:
metlife_web = dertyd.copy()

In [ ]:
metlife_web.head()
metlife_web.shape

In [27]:


# nuevos = "F:/Empresas SF/reportes/carga.xlsx"



metlife_web["rut1"] = metlife_web["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut1"] = metlife_web["rut1"].str.replace("-", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace("–", "")
metlife_web["rut1"] = metlife_web["rut1"].astype(str)
metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
metlife_web["rut1"] = metlife_web["rut1"].str.replace(" ", "")
metlife_web["rut1"] = metlife_web["rut1"].str.upper()
metlife_web["rut1"] = metlife_web["rut1"].str.strip()

metlife_web["rut2"] = metlife_web["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].str.replace("-", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace("–", "")
metlife_web["rut2"] = metlife_web["rut2"].astype(str)
metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
metlife_web["rut2"] = metlife_web["rut2"].str.replace(" ", "")
metlife_web["rut2"] = metlife_web["rut2"].str.upper()
metlife_web["rut2"] = metlife_web["rut2"].str.strip()

metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(",", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace(".", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("-", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┐", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("═", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("╔", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.replace("┌", "")
metlife_web["Nombre"] = metlife_web["Nombre"].str.title()
metlife_web["Nombre"] = metlife_web["Nombre"].str.strip()

metlife_web["poliza"] = metlife_web["poliza"].astype(str)
metlife_web["poliza"] = metlife_web["poliza"].str.strip()

metlife_web["rut1"] = metlife_web["rut1"].apply(lambda y: siPrimerCero(y))
metlife_web["rut2"] = metlife_web["rut2"].apply(lambda y: siPrimerCero(y))
metlife_web = metlife_web[(metlife_web["rut1"] != "NONE")]
metlife_web = metlife_web[(metlife_web["rut2"] != "NONE")]
metlife_web.shape

metlife_web["Titular"] = np.where(metlife_web["rut1"] == metlife_web["rut2"],1,2 )
metlife_web = metlife_web[(metlife_web["Titular"] == 1)]

metlife_web = metlife_web.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_web.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\2932449918.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut1"] = metlife_web["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\2932449918.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["rut2"] = metlife_web["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\2932449918.py:26: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_web["Nombre"] = metlife_web["Nombre"].

(12274, 39)

In [28]:
webq = "F:/2024/MetLife/" + str(mesesDic[mes]) + año + "/Subir/Q.xlsx"
metlife_Q = pd.read_excel(webq)

metlife_Q["rut1"] = metlife_Q["rut1"].astype(str)
#metlife_sftp["rut1"] = metlife_sftp["rut1"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace("-", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace("–", "")
metlife_Q["rut1"] = metlife_Q["rut1"].astype(str)
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(".", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(" ", "")
metlife_Q["rut1"] = metlife_Q["rut1"].str.upper()
metlife_Q["rut1"] = metlife_Q["rut1"].str.strip()

metlife_Q["rut2"] = metlife_Q["rut2"].astype(str)
#metlife_sftp["rut2"] = metlife_sftp["rut2"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace("-", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace("–", "")
metlife_Q["rut2"] = metlife_Q["rut2"].astype(str)
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(".", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(" ", "")
metlife_Q["rut2"] = metlife_Q["rut2"].str.upper()
metlife_Q["rut2"] = metlife_Q["rut2"].str.strip()

metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace(",", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace(".", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("-", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("┐", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("═", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("╔", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("┌", "")
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.title()
metlife_Q["Nombre"] = metlife_Q["Nombre"].str.strip()

metlife_Q["poliza"] = metlife_Q["poliza"].astype(str)
metlife_Q["poliza"] = metlife_Q["poliza"].str.strip()

metlife_Q["rut1"] = metlife_Q["rut1"].apply(lambda y: siPrimerCero(y))
metlife_Q["rut2"] = metlife_Q["rut2"].apply(lambda y: siPrimerCero(y))
metlife_Q = metlife_Q[(metlife_Q["rut1"] != "NONE")]
metlife_Q = metlife_Q[(metlife_Q["rut2"] != "NONE")]
metlife_Q.shape

metlife_Q["Titular"] = np.where(metlife_Q["rut1"] == metlife_Q["rut2"],1,2 )
metlife_Q = metlife_Q[(metlife_Q["Titular"] == 1)]

metlife_Q = metlife_Q.drop_duplicates(
    [
        "rut1","poliza"
    ],
    keep="last",
)


#*******************sacar primer cero
metlife_Q.shape
#derty = pd.read_excel(sftp)

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\3947619704.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["rut1"] = metlife_Q["rut1"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\3947619704.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["rut2"] = metlife_Q["rut2"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_16476\3947619704.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  metlife_Q["Nombre"] = metlife_Q["Nombre"].str.replace("

(326268, 25)

In [29]:
subir_correo = pd.DataFrame()
subir_correo["poliza_rut"] = metlife_correo["poliza"]
subir_correo["rut_rut"] = metlife_correo["rut1"]
subir_correo["nombre_rut"] = metlife_correo["Nombre"]
subir_correo["mes_rut"] = str(mes)
subir_correo["año_rut"] = str(año)
#subir_correo["fecha_rut"] = metlife_correo["fecha"].apply(lambda y: fechaValida(y))
subir_correo["Llave"] = metlife_correo["poliza"] + metlife_correo["rut1"] + str(mes) + str(año)
subir_correo["desde_rut"] = metlife_correo["desde"].apply(lambda y: fechaValida(y))
subir_correo["hasta_rut"] = metlife_correo["hasta"].apply(lambda y: fechaValida(y))
subir_correo["canal_rut"] = "email"

subir_sql = pd.DataFrame()
subir_sql["poliza_rut"] = metlife_sftp["poliza"]
subir_sql["rut_rut"] = metlife_sftp["rut1"]
subir_sql["nombre_rut"] = metlife_sftp["Nombre"]
subir_sql["mes_rut"] = str(mes)
subir_sql["año_rut"] = str(año)
#subir_sql["fecha_rut"] = metlife_sftp["fecha"].apply(lambda y: fechaValida(y))
subir_sql["Llave"] = metlife_sftp["poliza"] + metlife_sftp["rut1"] + str(mes) + str(año)
subir_sql["desde_rut"] = metlife_sftp["desde"].apply(lambda y: fechaValida(y))
subir_sql["hasta_rut"] = metlife_sftp["hasta"].apply(lambda y: fechaValida(y)) 
subir_sql["canal_rut"] = "sftp"

subir_web = pd.DataFrame()
subir_web["poliza_rut"] = metlife_web["poliza"]
subir_web["rut_rut"] = metlife_web["rut1"]
subir_web["nombre_rut"] = metlife_web["Nombre"]
subir_web["mes_rut"] = str(mes)
subir_web["año_rut"] = str(año)
#subir_web["fecha_rut"] = metlife_web["fecha"].apply(lambda y: fechaValida(y))
subir_web["Llave"] = metlife_web["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_web["desde_rut"] = metlife_web["desde"].apply(lambda y: fechaValida(y))
subir_web["hasta_rut"] = metlife_web["hasta"].apply(lambda y: fechaValida(y))
subir_web["canal_rut"] = "web"

subir_Q = pd.DataFrame()
subir_Q["poliza_rut"] = metlife_Q["poliza"]
subir_Q["rut_rut"] = metlife_Q["rut1"]
subir_Q["nombre_rut"] = metlife_Q["Nombre"]
subir_Q["mes_rut"] = str(mes)
subir_Q["año_rut"] = str(año)
#subir_Q["fecha_rut"] = metlife_Q["fecha"].apply(lambda y: fechaValida(y))
subir_Q["Llave"] = metlife_Q["poliza"] + metlife_web["rut1"] + str(mes) + str(año)
subir_Q["desde_rut"] = metlife_Q["desde"].apply(lambda y: fechaValida(y))
subir_Q["hasta_rut"] = metlife_Q["hasta"].apply(lambda y: fechaValida(y))
subir_Q["canal_rut"] = "correo2"

mes_metlife = pd.DataFrame()
mes_metlife = pd.concat(
    [
        subir_web,
        subir_sql,
        subir_correo,
        subir_Q,
        
    ],
    axis=0,
)
mes_metlife["fecha_rut"] = pd.to_datetime(fecha, yearfirst=True)

mes_metlife = mes_metlife.drop_duplicates(
    [
        "rut_rut","poliza_rut",
    ],
    keep="last",
)
mes_metlife.head()

,poliza_rut,rut_rut,nombre_rut,mes_rut,año_rut,Llave,desde_rut,hasta_rut,canal_rut,fecha_rut
64,340002405,184584204,Yenifer Gallardo Maichil,4,2024,34000240518458420442024,2024-04-08 14:23:00,2024-04-08 14:23:00,web,2024-04-01
115,340002405,199950312,Josefa Cuevas Brito,4,2024,34000240519995031242024,2024-04-08 14:23:00,2024-04-08 14:23:00,web,2024-04-01
138,340002436,175044426,Patricio Quezada Habert,4,2024,34000243617504442642024,2024-04-08 14:23:00,2024-04-08 14:23:00,web,2024-04-01
142,340002460,196382976,Alberto Villavicencio Tagle,4,2024,34000246019638297642024,2024-04-08 14:23:00,2024-04-08 14:23:00,web,2024-04-01
145,340002552,117662500,Guillermo Pereira Gavilan,4,2024,34000255211766250042024,2024-04-08 14:23:00,2024-04-08 14:23:00,web,2024-04-01


//////////////////////////////////////////////////////////////////////////

NUMERO RUTS POR POLIZA
****************************************************************

In [30]:
mes_metlife.shape
print ( str(mes) + "-" + str(año) + " " + str(mes_metlife.shape[0]))

4-2024 329204


////////////////////////////////////////////////

NUMERO RUT UNICOS
------------------------------------------------------------------------------------------------

In [31]:
mes_metlife2 = mes_metlife.drop_duplicates(
    [
        "rut_rut"
    ],
    keep="last",
)
mes_metlife2.shape
print (str(mes) + "-" + str(año) + " " + str(mes_metlife2.shape[0]))

4-2024 326571


In [32]:
#subir_web["Llave"]
mes_metlife["esta"] =  mes_metlife["Llave"].isin(rut_unico_tiempo_["Llave"])
mes_metlife = mes_metlife[(mes_metlife["esta"] == False)]

mes_metlife = mes_metlife[
    [
        "poliza_rut",
        "rut_rut",
        "nombre_rut",
        "mes_rut",
        "año_rut",
        "fecha_rut",
        "Llave",
        "desde_rut",
        "hasta_rut",
        "canal_rut"
    ]
]
mes_metlife.shape

(329204, 10)

In [33]:
mes_metlife.to_sql(
    name="rut_unico_tiempo_2",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

329204

In [34]:
subir_unicos = pd.DataFrame()
subir_unicos["Rut_T"] = mes_metlife2["rut_rut"]
subir_unicos["Rut_C"] = mes_metlife2["rut_rut"]
subir_unicos["Nombre"] = mes_metlife2["nombre_rut"]
subir_unicos["Llave"] = mes_metlife2["rut_rut"] +  mes_metlife2["rut_rut"]


subir_unicos.shape

(326571, 4)

In [35]:
subir_unicos["esta"] =  subir_unicos["Rut_T"].isin(rut_unicos["Rut_T"])
subir_unicos = subir_unicos[(subir_unicos["esta"] == False)]

subir_unicos = subir_unicos[
    [
        "Rut_T",
        "Rut_C",
        "Nombre",
        "Llave",
    ]
]
subir_unicos.shape
subir_unicos.to_sql(
    name="rut_unicos",
    con=motor_mysql_mariadb,
    if_exists="append",
    index=False,
    chunksize=10000,
)

12699